Import Package

In [173]:
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

Import Data

In [ ]:
df = pd.read_excel('Online Retail.xlsx')

In [ ]:
df.head(5)

In [ ]:
df.dtypes

Cleansing Data

In [ ]:
# Clean up spaces in description and remove any rows that don't have a valid invoice
df['Description'] = df['Description'].str.strip() #hapus spasi dalam desc
df.dropna(axis=0, subset=['InvoiceNo'], inplace=True)# menghapus data yg invoicenya =0

In [ ]:
df['InvoiceNo'] = df['InvoiceNo'].astype('str')#convert semua data di InvoiceNo -> String
df = df[~df['InvoiceNo'].str.contains('C')]#hapus data yg menghapus pembayaran kredit dengan kode C

In [ ]:
df.head(5)

In [ ]:
basket = (df[df['Country'] =="France"]#flter data yg dr France
          .groupby(['InvoiceNo','Description'])['Quantity'] #group by 
          .sum().unstack().reset_index().fillna(0)
          .set_index('InvoiceNo'))#mengganti value index menjadi value dr Invoice No

In [ ]:
basket.head(5)

In [ ]:
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

basket_sets = basket.applymap(encode_units)
basket_sets.drop('POSTAGE', inplace=True, axis=1)

In [ ]:
basket_sets.head(5)

In [ ]:
frequent_itemsets = apriori(basket_sets, min_support=0.07, use_colnames=True)

In [ ]:
frequent_itemsets

In [ ]:
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=0.8)

In [ ]:
#rules1 = rules[("antecedents","consequents","support","confidence","lift")]
rules1 = rules[['antecedents', 'consequents', 'support','confidence','lift']]

In [ ]:
rules1.head()

In [ ]:
rules1[ (rules['lift'] >= 6) &
       (rules['confidence'] >= 0.8) ]

In [ ]:
strong_rules = rules[ (rules['lift'] >= 6) &
       (rules['confidence'] >= 0.8) ]


item = input('Item : ')
strong_rules[strong_rules['antecedents'].astype(str).str.contains(item.upper())].iloc[:,[0,1]]

In [ ]:
basket2 = (df[df['Country'] =="Germany"]
          .groupby(['InvoiceNo', 'Description'])['Quantity']
          .sum().unstack().reset_index().fillna(0)
          .set_index('InvoiceNo'))

basket_sets2 = basket2.applymap(encode_units)
basket_sets2.drop('POSTAGE', inplace=True, axis=1)
frequent_itemsets2 = apriori(basket_sets2, min_support=0.05, use_colnames=True)
rules2 = association_rules(frequent_itemsets2, metric="lift", min_threshold=1)
rules2 = rules2[['antecedents', 'consequents', 'support','confidence','lift']]

rules2[ (rules2['lift'] >= 4) &
        (rules2['confidence'] >= 0.5)]